In [1]:
import os
import json
import pandas as pd
import numpy as np
import json

from matplotlib import pyplot as plt
import seaborn as sb

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [2]:
sb.set_style("darkgrid")
sb.set_context("paper",  font_scale = 1,
               rc={
                   "grid.linewidth": 3.75,
                   'axes.labelsize': 28,
                   "axes.titlesize": 32,
                   "legend.fontsize": 28.0,
                   'lines.linewidth': 5,
                   'xtick.labelsize': 24.0,
                   'ytick.labelsize': 24.0,
               })
current_palette = list(sb.color_palette())
colors = {
    'RL': current_palette[0],
    'RR': current_palette[5],
    'SOP':current_palette[4],
    'RAND':current_palette[7],
    'optimal':current_palette[1],
    '0':current_palette[-2],
    '1':current_palette[-1]
}

In [3]:
experiments = ['grid', 'psrm', 'driverlog', 'trucks']

In [69]:
# Load the validated RL data
exp_dir = '/home/biedenka/git/rl-plan'
rl_data = {}
rl_time_data = {}
UNITS = 75
cutoff = 7500  # used cutoff for all experiments

epm = 'grid'

for epm in experiments:
    data_dir = "/mhome/biedenka/rl-plan-tmp/experiments/%s" % epm
    rl_data[epm] = {}
    print(epm)

    for TEST in [True, False]:  # For all trining and test splits
#         print('TEST ', TEST)
        insts = sorted(os.listdir(os.path.join(data_dir, 'test' if TEST else 'train', epm)))
        rl_cost = {}
        rl_traj = {}
        rl_tims = {}
        rl_times = {}
        timeouts = 0
        means = []
        trajs = []
        timouts = []
        times = []
        for seed in range(1, 6):  # for all 5 seeds
            try:
                file=os.path.join(
                    data_dir,
                    '%s_full_train_set_2-%d-%d' % (
                        epm, UNITS, seed),
                    'best',
                    '%s-2-%d-seed%d.json' % (
                        'test' if TEST else 'train', UNITS, seed))
    #             print(file)
                with open(file, 'r') as fh:
                    data = json.load(fh)
    #             print(data)
                timeouts = 0
                for traj in data['trajectories']:  # load the trajectories and see if they match the cutoff
        #             print(len(traj))
                    if len(traj) >= cutoff:
                        timeouts += 1
                means.append(data['mean'])         # load the average plan cost over all 22/24 training or 8/6 test instances
                trajs.append(data['trajectories']) # load all policies for runs on all 22/24 / 8/6 instances
                timouts.append(timeouts)
                times.append(data['times_per_episode'])
            except FileNotFoundError:
                print('Not done with seed %d' % (seed))

    #     # Compute the mean and stdev over 10 repetitions
        rl_cost = (np.mean(means), np.std(means))
        rl_traj = np.array(trajs)
        rl_tims = np.mean(timouts)
        rl_times = times

#         print('{:^7s} | {:^7s} | {:s}'.format('Mean', 'Stdev', 'Timeouts'))
#         print('{:>+7.2f} | {:>+7.2f} | {:>1.1f}'.format(*rl_cost,
#                                                         rl_tims))
#         print('#'*90)

#         print('Done')

        key = 'test' if TEST else 'train'
        rl_data[epm][key] = {}
        for inst_id, inst in enumerate(insts):
            plan_lens = []
            for seed_traj in rl_traj:
                plan_lens.append(len(seed_traj[inst_id]))
            rl_data[epm][key][inst] = np.mean(plan_lens)
        print('\t', key)
        for (k, v) in zip(rl_data[epm][key].keys(), rl_data[epm][key].values()):
            print('\t', k, v)
        print('\t\tMean: ', np.around(np.mean(list(rl_data[epm][key].values())), decimals=3))
        print('-'*80)
    print('#'*80)

grid
	 test
	 prob02.pddl 49.6
	 prob03.pddl 7500.0
		Mean:  3774.8
--------------------------------------------------------------------------------
	 train
	 prob01.pddl 31.8
	 prob04.pddl 64.6
	 prob05.pddl 454.6
		Mean:  183.667
--------------------------------------------------------------------------------
################################################################################
psrm
	 test
	 p06-s37-n3-l3-f30.pddl 27.8
	 p08-s40-n3-l4-f10.pddl 34.8
	 p15-s56-n4-l4-f10.pddl 23.8
	 p21-s72-n5-l4-f10.pddl 55.0
	 p27-s86-n6-l3-f50.pddl 43.2
	 p28-s88-n6-l4-f10.pddl 18.4
	 p33-s100-n7-l3-f10.pddl 520.8
	 p35-s103-n7-l3-f70.pddl 9.2
	 p37-s108-n7-l5-f10.pddl 111.0
	 p39-s112-n8-l2-f10.pddl 29.2
	 p41-s116-n8-l3-f10.pddl 39.4
	 p49-s149-n10-l3-f30.pddl 20.6
	 p50-s153-n10-l4-f30.pddl 89.8
		Mean:  78.692
--------------------------------------------------------------------------------
	 train
	 p01-s17-n2-l2-f30.pddl 8.4
	 p02-s23-n2-l3-f70.pddl 4.0
	 p03-s28-n2-l5-f10.pddl 20.2
	

<BR><BR><BR><BR><BR><BR><BR>

In [5]:
# Load the validated RR data

import itertools
perms = list(itertools.permutations([0, 1, 2]))

exp_dir = '/home/biedenka/git/rl-plan'
rr_data = {}

p = perms[0]

for epm in experiments:
    perm_instance_data = {
        'train': {
            0: {},
            1: {},
            2: {},
            3: {},
            4: {},
            5: {},
        },
        'test': {
            0: {},
            1: {},
            2: {},
            3: {},
            4: {},
            5: {},
        }
    }
    perm_instance_data_time = {
        'train': {
            0: {},
            1: {},
            2: {},
            3: {},
            4: {},
            5: {},
        },
        'test': {
            0: {},
            1: {},
            2: {},
            3: {},
            4: {},
            5: {},
        }
    }
    print(epm, end=' ')
    data_dir = "/mhome/biedenka/rl-plan-tmp/experiments/%s" % epm
    rr_data[epm] = {}

    for TEST in [True, False]:  # For all trining and test splits
        insts = sorted(os.listdir(os.path.join(data_dir, 'test' if TEST else 'train', epm)))
#         print('TEST ', TEST)
        perm_id = 0
        key = 'test' if TEST else 'train'
        for p in perms:
#             print(perm_id, end=' ')
            file=os.path.join(
                data_dir,
                '%s-rr-%d-%d-%d-seed-.json' % (
                    'test' if TEST else 'train', p[0], p[1], p[2]))
    #             print(file)
            with open(file, 'r') as fh:
                data = json.load(fh)
            for inst_idx, inst in enumerate(insts):
                    perm_instance_data[key][perm_id][inst] = len(
                        data['trajectories'][inst_idx])
                    perm_instance_data_time[key][perm_id][inst] = data['times_per_episode'][inst_idx]
            perm_id += 1
#         print()

    best_permuatation = None
    best_perm_perf = np.inf
#     print(perm_instance_data)
    for perm_id in perm_instance_data['train']:
        perf = np.mean(list(perm_instance_data['train'][perm_id].values()))
        if perf < best_perm_perf:
            best_perm_perf = perf
            best_permuatation = perm_id
    rr_data[epm] = perm_instance_data
    rr_data[epm]['best_permutation'] = best_permuatation
#         print(perm_id, perms[perm_id], perf)
#     print()
#     print('\t', 'Best RR on TRAIN instances: ', perms[best_permuatation])
#     print('\t', 'Average Performance on TRAIN: ', best_perm_perf)
    print(' - best sequence', perms[best_permuatation])
    for key in ['test', 'train']:
        print('\t', key)
        for (k, v) in zip(
            rr_data[epm][key][rr_data[epm]['best_permutation']].keys(),
            rr_data[epm][key][rr_data[epm]['best_permutation']].values()):
            print('\t', k, v)
        print('\t\tMean: ', np.around(np.mean(list(rr_data[epm][key][rr_data[epm]['best_permutation']].values())), decimals=3))
        print('-'*80)
    print('#'*80)

grid  - best sequence (2, 0, 1)
	 test
	 prob02.pddl 41
	 prob03.pddl 7500
		Mean:  3770.5
--------------------------------------------------------------------------------
	 train
	 prob01.pddl 19
	 prob04.pddl 77
	 prob05.pddl 975
		Mean:  357.0
--------------------------------------------------------------------------------
################################################################################
psrm  - best sequence (2, 1, 0)
	 test
	 p06-s37-n3-l3-f30.pddl 25
	 p08-s40-n3-l4-f10.pddl 198
	 p15-s56-n4-l4-f10.pddl 1205
	 p21-s72-n5-l4-f10.pddl 2350
	 p27-s86-n6-l3-f50.pddl 105
	 p28-s88-n6-l4-f10.pddl 1712
	 p33-s100-n7-l3-f10.pddl 7500
	 p35-s103-n7-l3-f70.pddl 9
	 p37-s108-n7-l5-f10.pddl 7500
	 p39-s112-n8-l2-f10.pddl 114
	 p41-s116-n8-l3-f10.pddl 7500
	 p49-s149-n10-l3-f30.pddl 1700
	 p50-s153-n10-l4-f30.pddl 7500
		Mean:  2878.308
--------------------------------------------------------------------------------
	 train
	 p01-s17-n2-l2-f30.pddl 12
	 p02-s23-n2-l3-f70.pddl 3

<BR><BR><BR><BR><BR><BR><BR>

In [6]:
# Load the validated RAND data

import itertools
exp_dir = '/home/biedenka/git/rl-plan'
rand_data = {}
rand_data_time = {}

cutoff = 7500  # used cutoff for all experiments


for epm in experiments:
    print(epm)
    data_dir = "/mhome/biedenka/rl-plan-tmp/experiments/%s" % epm
    rand_data[epm] = {}
    rand_data_time[epm] = {}

    for TEST in [True, False]:  # For all trining and test splits
        insts = sorted(os.listdir(os.path.join(data_dir, 'test' if TEST else 'train', epm)))
        key = 'test' if TEST else 'train'
        rand_data[epm][key] = {}
        rand_data_time[epm][key] = {}
        for seed in range(1, 6):  # for all 10 seeds
            file=os.path.join(
                data_dir,
                '%s-rand-seed-%d.json' % (
                    'test' if TEST else 'train', seed))
#                 print(file)
            with open(file, 'r') as fh:
                data = json.load(fh)

            for inst_idx, inst in enumerate(insts):
                if inst not in rand_data:
                    rand_data[epm][key][inst] = []
                    rand_data_time[epm][key][inst] = []
                rand_data[epm][key][inst].append(len(
                    data['trajectories'][inst_idx]))
                rand_data_time[epm][key][inst].append(
                    data['times_per_episode'][inst_idx])

        for inst in rand_data[epm][key]:
            rand_data[epm][key][inst] = np.mean(rand_data[epm][key][inst])
            rand_data_time[epm][key][inst] = np.mean(rand_data_time[epm][key][inst])
        print('\t', key)
        for (k, v) in zip(rand_data[epm][key].keys(), rand_data[epm][key].values()):
            print('\t', k, v)
        print('\t\tMean: ', np.around(np.mean(list(rand_data[epm][key].values())), decimals=3))
        print('-'*80)
    print('#'*80)

grid
	 test
	 prob02.pddl 32.0
	 prob03.pddl 7500.0
		Mean:  3766.0
--------------------------------------------------------------------------------
	 train
	 prob01.pddl 19.0
	 prob04.pddl 96.0
	 prob05.pddl 6629.0
		Mean:  2248.0
--------------------------------------------------------------------------------
################################################################################
psrm
	 test
	 p06-s37-n3-l3-f30.pddl 22.0
	 p08-s40-n3-l4-f10.pddl 215.0
	 p15-s56-n4-l4-f10.pddl 206.0
	 p21-s72-n5-l4-f10.pddl 2978.0
	 p27-s86-n6-l3-f50.pddl 97.0
	 p28-s88-n6-l4-f10.pddl 2243.0
	 p33-s100-n7-l3-f10.pddl 7500.0
	 p35-s103-n7-l3-f70.pddl 9.0
	 p37-s108-n7-l5-f10.pddl 7500.0
	 p39-s112-n8-l2-f10.pddl 225.0
	 p41-s116-n8-l3-f10.pddl 5374.0
	 p49-s149-n10-l3-f30.pddl 7500.0
	 p50-s153-n10-l4-f30.pddl 7500.0
		Mean:  3182.231
--------------------------------------------------------------------------------
	 train
	 p01-s17-n2-l2-f30.pddl 9.0
	 p02-s23-n2-l3-f70.pddl 4.0
	 p03-s28-n2-l

In [10]:
# Load the validated STATIC data

import itertools
exp_dir = '/home/biedenka/git/rl-plan'
static_data = {}
static_data_time = {}

cutoff = 7500  # used cutoff for all experiments

for epm in experiments:
    print(epm, end=' ')
    data_dir = "/mhome/biedenka/rl-plan-tmp/experiments/%s" % epm
    static_data[epm] = {}
    static_data_time[epm] = {}
    
    best_action, best_perf = None, np.inf
    oracle_portfolio = {}
    for action in range(3):  # for all individual actions
        try:
            static_data[epm][action] = {}
            static_data_time[epm][action] = {}
            for TEST in [True, False]:  # For all trining and test splits
                insts = sorted(os.listdir(
                    os.path.join(data_dir, 'test' if TEST else 'train', epm)))
                key = 'test' if TEST else 'train'
                static_data[epm][action][key] = {}
                static_data_time[epm][action][key] = {}
                file=os.path.join(
                    data_dir,
                    '%s-static-%d.json' % (
                        'test' if TEST else 'train', action))
    #                 print(file)
                with open(file, 'r') as fh:
                    data = json.load(fh)

                for inst_idx, inst in enumerate(insts):
                    if inst not in static_data:
                        static_data[epm][action][key][inst] = []
                        static_data_time[epm][action][key][inst] = []
                    static_data[epm][action][key][inst].append(len(
                        data['trajectories'][inst_idx]))
                    static_data_time[epm][action][key][inst].append(
                        data['times_per_episode'][inst_idx])

                for inst in static_data[epm][action][key]:
                    static_data[epm][action][key][inst] = np.mean(
                        static_data[epm][action][key][inst])
                    static_data_time[epm][action][key][inst] = np.mean(
                        static_data_time[epm][action][key][inst])
                    if inst not in oracle_portfolio:
                        oracle_portfolio[inst] = np.mean(
                        static_data[epm][action][key][inst])
                    elif static_data[epm][action][key][inst] < oracle_portfolio[inst]:
                        oracle_portfolio[inst] = static_data[epm][action][key][inst]
            average_on_train = np.mean(
                list(static_data[epm][action]['train'].values())
                )
            if average_on_train < best_perf:
                best_action = action
                best_perf = average_on_train
        except:
            print('Action %d not done' % action)
    static_data[epm]['best_action'] = best_action
    static_data[epm]['oracle'] = oracle_portfolio
    print('- best action: %d' % best_action)
    for key in ['test', 'train']:
        print('\t', key)
        for (k, v) in zip(static_data[epm][best_action][key].keys(), 
                          static_data[epm][best_action][key].values()):
            print('\t', k, v)
        print('\t\tMean: ', np.around(
            np.mean(list(static_data[epm][best_action][key].values())), decimals=3))
        print('-'*80)
    print('#'*80)

grid - best action: 1
	 test
	 prob02.pddl 201.0
	 prob03.pddl 7500.0
		Mean:  3850.5
--------------------------------------------------------------------------------
	 train
	 prob01.pddl 41.0
	 prob04.pddl 41.0
	 prob05.pddl 7500.0
		Mean:  2527.333
--------------------------------------------------------------------------------
################################################################################
psrm - best action: 2
	 test
	 p06-s37-n3-l3-f30.pddl 25.0
	 p08-s40-n3-l4-f10.pddl 33.0
	 p15-s56-n4-l4-f10.pddl 23.0
	 p21-s72-n5-l4-f10.pddl 54.0
	 p27-s86-n6-l3-f50.pddl 42.0
	 p28-s88-n6-l4-f10.pddl 18.0
	 p33-s100-n7-l3-f10.pddl 520.0
	 p35-s103-n7-l3-f70.pddl 9.0
	 p37-s108-n7-l5-f10.pddl 110.0
	 p39-s112-n8-l2-f10.pddl 28.0
	 p41-s116-n8-l3-f10.pddl 28.0
	 p49-s149-n10-l3-f30.pddl 19.0
	 p50-s153-n10-l4-f30.pddl 88.0
		Mean:  76.692
--------------------------------------------------------------------------------
	 train
	 p01-s17-n2-l2-f30.pddl 8.0
	 p02-s23-n2-l3-f70.pdd

In [8]:

print('Static Oracle Performance')
for epm in experiments:
    print(epm)
    for key in ['test', 'train']:
        print('\t', key)
        vals = []
        for inst in static_data[epm][0][key].keys():
            v = static_data[epm]['oracle'][inst]
            vals.append(v)
            print(inst, v)
        print('\t\tMean: ', np.around(np.mean(vals), decimals=3))
        print('-'*80)
    print('#'*80)

Static Oracle Performance
grid
	 test
prob02.pddl 34.0
prob03.pddl 5605.0
		Mean:  2819.5
--------------------------------------------------------------------------------
	 train
prob01.pddl 25.0
prob04.pddl 41.0
prob05.pddl 7500.0
		Mean:  2522.0
--------------------------------------------------------------------------------
################################################################################
psrm
	 test
p06-s37-n3-l3-f30.pddl 25.0
p08-s40-n3-l4-f10.pddl 33.0
p15-s56-n4-l4-f10.pddl 23.0
p21-s72-n5-l4-f10.pddl 54.0
p27-s86-n6-l3-f50.pddl 42.0
p28-s88-n6-l4-f10.pddl 18.0
p33-s100-n7-l3-f10.pddl 520.0
p35-s103-n7-l3-f70.pddl 7.0
p37-s108-n7-l5-f10.pddl 110.0
p39-s112-n8-l2-f10.pddl 28.0
p41-s116-n8-l3-f10.pddl 28.0
p49-s149-n10-l3-f30.pddl 19.0
p50-s153-n10-l4-f30.pddl 88.0
		Mean:  76.538
--------------------------------------------------------------------------------
	 train
p01-s17-n2-l2-f30.pddl 5.0
p02-s23-n2-l3-f70.pddl 3.0
p03-s28-n2-l5-f10.pddl 19.0
p04-s31-n2-l5-f70

In [71]:
from IPython.display import display, HTML

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

choice = {
    'RL': rl_data,
    'RR': rr_data,
    'RAND': rand_data,
    'BS': static_data,
    'SOP': static_data
}
header=list(choice.keys())
for epm in experiments:
    print(epm)
    all_data = {}
    for key in ['test', 'train']:
        print(key)
        data = {}
        for inst in rl_data[epm][key]:
            data[inst] = [rl_data[epm][key][inst]]
            for act in range(6):
                data[inst].append(
    #                 rr_data[epm][key][rr_data[epm]['best_permutation']][inst]
                    rr_data[epm][key][act][inst]
                )
            data[inst].append(rand_data[epm][key][inst])
            data[inst].append(
                static_data[epm][static_data[epm]['best_action']][key][inst])
            data[inst].append(static_data[epm]['oracle'][inst])
            all_data[inst] = data[inst]
        df = pd.DataFrame.from_dict(data, orient='index',
                                    columns=['RL', 'RR-0', 'RR-1', 'RR-2', 'RR-3',
                                             'RR-4', 'RR-5', 'RAND', 'BS', 'SOP'])
#         df = df.replace(7500, np.nan)
        df.loc['mean'] = df.mean()
        display(df)
        if key == 'test':
            print()
            print('#'*80)
        print()
#     print('~'*80)
#     print()
    
#     df = pd.DataFrame.from_dict(all_data, orient='index',
#                                 columns=['RL', 'RR-0', 'RR-1', 'RR-2', 'RR-3',
#                                          'RR-4', 'RR-5', 'RAND', 'BS', 'SOP'])
#     df.loc['mean'] = df.mean()
#     display(df)
    print()
    print('='*80)
    print('='*80)
    print()

grid
test


,RL,RR-0,RR-1,RR-2,RR-3,RR-4,RR-5,RAND,BS,SOP
prob02.pddl,49.60,50.00,35.00,45.00,49.00,41.00,46.00,32.00,201.00,34.00
prob03.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,5605.00
mean,3774.80,3775.00,3767.50,3772.50,3774.50,3770.50,3773.00,3766.00,3850.50,2819.50



################################################################################

train


,RL,RR-0,RR-1,RR-2,RR-3,RR-4,RR-5,RAND,BS,SOP
prob01.pddl,31.80,22.00,19.00,19.00,20.00,19.00,18.00,19.00,41.00,25.00
prob04.pddl,64.60,106.00,91.00,77.00,78.00,77.00,102.00,96.00,41.00,41.00
prob05.pddl,454.60,974.00,7500.00,7500.00,7500.00,975.00,7500.00,6629.00,7500.00,7500.00
mean,183.67,367.33,2536.67,2532.00,2532.67,357.00,2540.00,2248.00,2527.33,2522.00





psrm
test


...

,RL,RR-0,RR-1,RR-2,RR-3,RR-4,RR-5,RAND,BS,SOP
p01.pddl,8.00,9.00,8.00,8.00,8.00,8.00,9.00,8.00,8.00,8.00
p02.pddl,25.40,32.00,28.00,35.00,31.00,38.00,32.00,26.00,24.00,24.00
p03.pddl,13.40,13.00,14.00,16.00,13.00,15.00,13.00,13.00,15.00,13.00
p04.pddl,20.80,41.00,28.00,26.00,28.00,25.00,38.00,43.00,27.00,20.00
p05.pddl,31.20,22.00,28.00,22.00,22.00,21.00,23.00,20.00,21.00,21.00
p06.pddl,15.60,17.00,19.00,13.00,18.00,13.00,17.00,18.00,17.00,13.00
p08.pddl,58.60,37.00,48.00,44.00,36.00,38.00,45.00,29.00,38.00,38.00
p09.pddl,38.40,43.00,41.00,75.00,31.00,44.00,40.00,39.00,28.00,28.00
p11.pddl,49.40,69.00,73.00,42.00,42.00,84.00,75.00,31.00,39.00,34.00
p12.pddl,129.20,122.00,124.00,125.00,124.00,126.00,126.00,84.00,196.00,100.00





trucks
test


,RL,RR-0,RR-1,RR-2,RR-3,RR-4,RR-5,RAND,BS,SOP
p01.pddl,32.80,22.00,27.00,22.00,27.00,23.00,20.00,25.00,65.00,16.00
p02.pddl,136.80,63.00,412.00,413.00,411.00,413.00,61.00,377.00,147.00,60.00
p03.pddl,209.60,117.00,737.00,514.00,617.00,48.00,515.00,103.00,298.00,298.00
p05.pddl,91.60,151.00,151.00,111.00,179.00,119.00,144.00,167.00,85.00,85.00
p06.pddl,2122.00,113.00,7500.00,7500.00,112.00,7500.00,7500.00,81.00,1179.00,1179.00
p07.pddl,1301.20,7500.00,1260.00,1258.00,1258.00,1257.00,7500.00,656.00,592.00,592.00
p08.pddl,675.20,1572.00,1603.00,1448.00,351.00,6844.00,2447.00,1598.00,357.00,215.00
p09.pddl,7500.00,7500.00,7500.00,5729.00,7500.00,3376.00,7500.00,7500.00,7500.00,1836.00
p10.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p11.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00



################################################################################

train


,RL,RR-0,RR-1,RR-2,RR-3,RR-4,RR-5,RAND,BS,SOP
p04.pddl,124.40,4572.00,3219.00,3325.00,573.00,3215.00,591.00,4192.00,223.00,223.00
p12.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p14.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p17.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p19.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p22.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p28.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
p30.pddl,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00,7500.00
mean,6578.05,7134.00,6964.88,6978.12,6634.12,6964.38,6636.38,7086.50,6590.38,6590.38


In [54]:
print(rr_data['driverlog']['test'].keys())

dict_keys([0, 1, 2, 3, 4, 5])
